In [1]:
import os

os.chdir("..")

In [2]:
import random
from typing import List

from PIL import Image
import matplotlib.pyplot as plt

import os
import filetype

import numpy as np

from scripts.utils import save_dict_to_json


def get_image_paths(dire, max_num=10):
    fps = [os.path.join(dire, f) for f in 
           sorted(os.listdir(dire), key=lambda x: int(x.split(".")[0]))]
    fps = [f for f in fps if filetype.is_image(f)]
    return sorted(fps)[:max_num]


def get_n_random_order_lists(n: int, m: int) -> List[List[int]]:
    """
    Generate n random order lists of length m.
    """
    return [random.sample(range(m), m) for _ in range(n)]


def get_2_random_order_lists(m: int) -> List[List[int]]:
    """
    Generate 2 random order lists of length m.
    """
    return get_n_random_order_lists(2, m)


def get_reordering_indices(target_list: list[int], shuffled_list: list[int]) -> list[int]:
    """
    Get the reordering indices to transform shuffled_list to target_list.
    
    Both the target list and the shuffled list can only contain unique values.
    """
    assert len(target_list) == len(shuffled_list), "Lists must be of the same length"
    assert len(set(target_list)) == len(target_list), "Target list must contain unique values"
    assert len(set(shuffled_list)) == len(shuffled_list), "Shuffled list must contain unique values"
    assert set(target_list) == set(shuffled_list), "Lists must contain the same elements"
    
    return [shuffled_list.index(x) for x in target_list]


def reodering(lst: list, indices: list[int]) -> list[int]:
    """
    Reorder the list lst according to the order specified by indices.
    """
    return [lst[i] for i in indices]


def resize_and_pad(img, size, pad_color=(255, 255, 255)):
    img.thumbnail(size, Image.LANCZOS)
    new_img = Image.new("RGB", size, pad_color)
    offset = ((size[0] - img.width) // 2, (size[1] - img.height) // 2)
    new_img.paste(img, offset)
    return new_img

def matplotlib_image_grid(img_paths, grid_size, img_size=(400, 400), 
                          font_size=14, line_color='black', save_path=None):
    rows, cols = grid_size
    assert len(img_paths) <= rows * cols, "Not enough space in grid for all images"

    fig_width = cols * (img_size[0]/100)
    fig_height = rows * (img_size[1]/100)

    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))

    # Normalize axes for different grid sizes
    if rows == 1 and cols == 1:
        axes = [[axes]]
    elif rows == 1 or cols == 1:
        axes = axes.reshape(rows, cols)

    for idx, (ax, img_path) in enumerate(zip(axes.flatten(), img_paths)):
        img = Image.open(img_path).convert('RGB')
        img = resize_and_pad(img, img_size)
        ax.imshow(img)
        ax.axis('off')

        # Add numbering
        ax.text(0.05, 0.95, str(idx + 1), fontsize=font_size, color='yellow',
                ha='left', va='top', transform=ax.transAxes,
                bbox=dict(facecolor='black', alpha=0.5, pad=3))

    # Fill remaining grid cells with blank images
    for i in range(len(img_paths), len(axes.flatten())):
        ax = axes.flatten()[i]
        ax.imshow(np.ones((img_size[1], img_size[0], 3), dtype=np.uint8) * 255)  # White placeholder
        ax.axis('off')

    plt.subplots_adjust(wspace=0.02, hspace=0.02)

    # Grid lines
    for r in range(1, rows):
        fig.add_artist(plt.Line2D([0, 1], [r/rows, r/rows], color=line_color, 
                                  linewidth=2, transform=fig.transFigure))
    for c in range(1, cols):
        fig.add_artist(plt.Line2D([c/cols, c/cols], [0, 1], color=line_color, 
                                  linewidth=2, transform=fig.transFigure))

    if save_path:
        plt.tight_layout()
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.tight_layout()
        plt.show()


def create_objects_map(seed_images_dir, save_dir, first_n=13,
                       grid_size=(3, 5), img_size=(400, 400),
                       font_size=14, line_color='black',
                       number_of_random_grids=10):
    
    os.makedirs(save_dir, exist_ok=True)

    fns = sorted(os.listdir(seed_images_dir), 
                 key=lambda x: int(x.split(".")[0]))
    fps = [os.path.join(seed_images_dir, f) for f in fns][:first_n]
    
    id_indices = list(range(1, len(fps) + 1))
    save_fp = os.path.join(save_dir, "id_order.png")
    matplotlib_image_grid(fps, grid_size, img_size, 
                          font_size, line_color, 
                          save_path=save_fp)
    mapper = {"data": {save_fp: id_indices,},  
              "metadata": {"grid_size": grid_size,
                           "img_size": img_size,
                           "font_size": font_size,
                           "line_color": line_color}}

    count = 0
    seen = set()
    seen.add(tuple(id_indices))
    while number_of_random_grids > count:
        count += 1

        while True:
            random_order = random.sample(id_indices, len(id_indices))
            reordering_indices = get_reordering_indices(id_indices, random_order)
            if tuple(reordering_indices) not in seen:
                break

        seen.add(tuple(reordering_indices))
        save_fp = os.path.join(save_dir, f"random_order_{count}.png")
        matplotlib_image_grid(reodering(fps, reordering_indices), grid_size, img_size, 
                              font_size, line_color, save_path=save_fp)
        mapper["data"][save_fp] = [x + 1 for x in reordering_indices]
    
    num_of_images = len(mapper["data"])
    mapper["metadata"]["number_of_objects_per_image"] = len(id_indices)
    mapper["metadata"]["number_of_images"] = len(mapper["data"])
    save_dict_to_json(mapper, os.path.join(save_dir, "mapper.json"))
    print(f"Saved {num_of_images} images to {save_dir}")
    
    return mapper

In [3]:
mapper = create_objects_map("data/scan-baskets-segmented", "data/baskets-grid-10", 
                            first_n=10, grid_size=(2, 5), img_size=(400, 400),
                            font_size=16, line_color='black',
                            number_of_random_grids=30)
mapper

Saved 31 images to data/baskets-grid-10


{'data': {'data/baskets-grid-10/id_order.png': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'data/baskets-grid-10/random_order_1.png': [8, 10, 4, 9, 2, 7, 1, 3, 6, 5],
  'data/baskets-grid-10/random_order_2.png': [9, 5, 6, 7, 3, 4, 1, 8, 10, 2],
  'data/baskets-grid-10/random_order_3.png': [7, 9, 6, 1, 3, 10, 8, 2, 5, 4],
  'data/baskets-grid-10/random_order_4.png': [1, 9, 3, 7, 10, 5, 2, 6, 8, 4],
  'data/baskets-grid-10/random_order_5.png': [8, 9, 10, 5, 7, 6, 1, 4, 2, 3],
  'data/baskets-grid-10/random_order_6.png': [2, 8, 6, 10, 7, 1, 5, 3, 4, 9],
  'data/baskets-grid-10/random_order_7.png': [4, 6, 10, 7, 5, 8, 1, 9, 2, 3],
  'data/baskets-grid-10/random_order_8.png': [1, 10, 9, 4, 2, 8, 6, 7, 5, 3],
  'data/baskets-grid-10/random_order_9.png': [2, 1, 7, 3, 5, 10, 6, 8, 9, 4],
  'data/baskets-grid-10/random_order_10.png': [1, 9, 4, 8, 6, 10, 7, 2, 3, 5],
  'data/baskets-grid-10/random_order_11.png': [7, 6, 10, 5, 8, 3, 2, 4, 1, 9],
  'data/baskets-grid-10/random_order_12.png': [10, 7, 2, 9, 

In [5]:
mapper = create_objects_map("data/scan-dogs-segmented", "data/dogs-grid-10", 
                            first_n=10, grid_size=(2, 5), img_size=(400, 400),
                            font_size=16, line_color='black',
                            number_of_random_grids=30)
mapper

Saved 31 images to data/dogs-grid-10


{'data': {'data/dogs-grid-10/id_order.png': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'data/dogs-grid-10/random_order_1.png': [2, 3, 7, 1, 9, 8, 5, 4, 10, 6],
  'data/dogs-grid-10/random_order_2.png': [8, 5, 6, 3, 10, 9, 2, 4, 1, 7],
  'data/dogs-grid-10/random_order_3.png': [8, 9, 7, 3, 10, 1, 6, 4, 2, 5],
  'data/dogs-grid-10/random_order_4.png': [7, 5, 2, 9, 10, 6, 8, 4, 1, 3],
  'data/dogs-grid-10/random_order_5.png': [7, 4, 1, 2, 5, 6, 8, 10, 3, 9],
  'data/dogs-grid-10/random_order_6.png': [7, 3, 8, 6, 5, 4, 1, 9, 2, 10],
  'data/dogs-grid-10/random_order_7.png': [10, 1, 6, 3, 4, 9, 7, 5, 2, 8],
  'data/dogs-grid-10/random_order_8.png': [2, 5, 8, 10, 1, 9, 4, 6, 7, 3],
  'data/dogs-grid-10/random_order_9.png': [3, 2, 4, 5, 9, 1, 7, 6, 10, 8],
  'data/dogs-grid-10/random_order_10.png': [2, 8, 1, 4, 5, 9, 6, 10, 7, 3],
  'data/dogs-grid-10/random_order_11.png': [8, 3, 7, 4, 1, 10, 9, 5, 6, 2],
  'data/dogs-grid-10/random_order_12.png': [2, 8, 7, 4, 6, 5, 1, 10, 3, 9],
  'data/dogs-grid-10

In [15]:
mapper = create_objects_map("data/scan-baskets-segmented", "data/baskets-grid", 
                            first_n=13, grid_size=(3, 5), img_size=(400, 400),
                            font_size=16, line_color='black',
                            number_of_random_grids=30)
mapper

Saved 31 images to data/baskets-grid


{'data': {'data/baskets-grid/id_order.png': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13],
  'data/baskets-grid/random_order_1.png': [8,
   4,
   1,
   3,
   12,
   7,
   13,
   11,
   5,
   9,
   10,
   6,
   2],
  'data/baskets-grid/random_order_2.png': [4,
   13,
   10,
   8,
   9,
   11,
   5,
   1,
   2,
   7,
   12,
   6,
   3],
  'data/baskets-grid/random_order_3.png': [9,
   2,
   5,
   7,
   13,
   12,
   4,
   10,
   8,
   6,
   3,
   11,
   1],
  'data/baskets-grid/random_order_4.png': [7,
   8,
   13,
   5,
   9,
   1,
   6,
   4,
   12,
   11,
   10,
   2,
   3],
  'data/baskets-grid/random_order_5.png': [3,
   11,
   8,
   1,
   6,
   5,
   9,
   10,
   13,
   12,
   7,
   4,
   2],
  'data/baskets-grid/random_order_6.png': [8,
   9,
   4,
   11,
   10,
   7,
   6,
   2,
   13,
   12,
   5,
   3,
   1],
  'data/baskets-grid/random_order_7.png': [6,
   11,
   9,
   3,
   12,
   4,
   7,
   5,
   8,
   2,
   10,
   1,
   13],
  'data/basket

In [16]:
mapper = create_objects_map("data/scan-dogs-segmented", "data/dogs-grid", 
                            first_n=13, grid_size=(3, 5), img_size=(400, 400),
                            font_size=16, line_color='black',
                            number_of_random_grids=30)
mapper

Saved 31 images to data/dogs-grid


{'data': {'data/dogs-grid/id_order.png': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13],
  'data/dogs-grid/random_order_1.png': [6,
   4,
   1,
   8,
   12,
   5,
   13,
   11,
   10,
   3,
   7,
   2,
   9],
  'data/dogs-grid/random_order_2.png': [8,
   4,
   12,
   11,
   9,
   7,
   13,
   2,
   1,
   10,
   5,
   6,
   3],
  'data/dogs-grid/random_order_3.png': [4,
   3,
   6,
   1,
   13,
   10,
   9,
   8,
   12,
   5,
   7,
   11,
   2],
  'data/dogs-grid/random_order_4.png': [1,
   12,
   8,
   3,
   4,
   11,
   13,
   9,
   2,
   5,
   7,
   10,
   6],
  'data/dogs-grid/random_order_5.png': [9,
   3,
   11,
   8,
   13,
   5,
   1,
   4,
   2,
   7,
   12,
   10,
   6],
  'data/dogs-grid/random_order_6.png': [1,
   6,
   9,
   11,
   4,
   2,
   3,
   5,
   13,
   8,
   12,
   7,
   10],
  'data/dogs-grid/random_order_7.png': [6,
   9,
   13,
   11,
   1,
   5,
   8,
   7,
   10,
   4,
   2,
   12,
   3],
  'data/dogs-grid/random_order_8.png':